In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import rpy2.robjects as ro
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects import numpy2ri
pandas2ri.activate()
import pickle

# Create output directory
output_dir = "./data/processed/expression/autoencoder/"
os.makedirs(output_dir, exist_ok=True)

# Define data paths
data_path = "./data/processed/expression/readcounts_tmm_all/"
metadata_path = "./data/processed/attphe.pkl"

# Load metadata
with open(metadata_path, 'rb') as f:
    metadata = pickle.load(f)

# List all tissue files
tissue_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".pkl")]

# Define Autoencoder using Functional API
def build_autoencoder(input_dim):
    input_layer = keras.Input(shape=(input_dim,))
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dense(64, activation='relu')(x)
    latent = layers.Dense(32, activation='relu')(x)  # Latent space
    x = layers.Dense(64, activation='relu')(latent)
    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(input_dim, activation='linear')(x)

    autoencoder = keras.Model(inputs=input_layer, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Encoder model to extract latent space
    encoder = keras.Model(inputs=input_layer, outputs=latent)

    return autoencoder, encoder

# Function to process each tissue file
def process_tissue_autoencoder(tissue_file, metadata, min_samples=10):
    tissue_name = os.path.basename(tissue_file).replace(".pkl", "")

    # Load normalized read counts
    with open(tissue_file, 'rb') as f:
        normalized_counts = pickle.load(f)

    sample_ids = normalized_counts.columns
    attr_filtered = metadata[metadata['samp_id'].isin(sample_ids)]

    # Skip tissue if the sample size is smaller than the minimum threshold
    if len(sample_ids) < min_samples:
        print(f"Skipping tissue {tissue_name} due to small sample size ({len(sample_ids)} samples).")
        return

    if normalized_counts.shape[1] != attr_filtered.shape[0]:
        raise ValueError(f"Number of samples in {tissue_name} does not match metadata.")

    # Resave normalized counts as DataFrame (using pandas HDF5 as previously)
    resaved_tissue_file = os.path.join(data_path, f"{tissue_name}_resaved.h5")
    normalized_counts.to_hdf(resaved_tissue_file, key='normalized_counts')
    print(f"Resaved normalized counts for {tissue_name} to {resaved_tissue_file}")

    
    input_dim = normalized_counts.shape[1]
    scaler = StandardScaler()
    normalized_counts_scaled  = scaler.fit_transform(normalized_counts.T).T  # Normalize across samples

    autoencoder, encoder = build_autoencoder(input_dim)

    # Train the autoencoder
    autoencoder.fit(normalized_counts_scaled,  normalized_counts_scaled, epochs=50, batch_size=32)

    # Encode latent features
    latent_features = encoder.predict(normalized_counts_scaled)

    # Load limma package in R
    ro.r('library(limma)')
    
    numpy2ri.activate()
    
    # Transpose expression matrix so rows = samples, columns = genes
    counts_T = normalized_counts.T  # shape: (samples, genes)
    r_counts = ro.conversion.py2rpy(counts_T.to_numpy())  # 
    # Convert latent features
    r_covariates = ro.conversion.py2rpy(np.array(latent_features))

    print("Expression shape:", normalized_counts.shape)
    print("Latent shape:", latent_features.shape)

    # Call R function
    adjusted_expression_data = ro.r['removeBatchEffect'](r_counts, covariates=r_covariates)

    # Convert adjusted data back to pandas DataFrame and transpose it to original shape
    adjusted_expression_df = pd.DataFrame(np.array(adjusted_expression_data),
                                          index=counts_T.index,
                                          columns=counts_T.columns).T  # 
    
    # Save adjusted expression data to pickle file
    result_file = os.path.join(output_dir, f"{tissue_name}.pkl")
    with open(result_file, 'wb') as f:
        pickle.dump(adjusted_expression_df, f)

    print(f"Processed tissue: {tissue_name}")
    print(f"Dimensions of adjusted data: {adjusted_expression_df.shape}")

# Loop through each tissue file and process them
for tissue_file in tissue_files:
    process_tissue_autoencoder(tissue_file, metadata)


Resaved normalized counts for Brain-Hypothalamus to ./data/processed/expression/readcounts_tmm_all/Brain-Hypothalamus_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0126
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9878
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9708
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.9520
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.9296
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9021
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.8707
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.8382
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.8076
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.7797
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.7541
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.7282
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.7014
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 202 probe(s) 



Expression shape: (14, 803)
Latent shape: (14, 32)
Coefficients not estimable: 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 3 12 
Processed tissue: Muscle-Skeletal
Dimensions of adjusted data: (14, 803)
Resaved normalized counts for Minor Salivary Gland to ./data/processed/expression/readcounts_tmm_all/Minor Salivary Gland_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0221
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9973
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9831
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.9724
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.9622
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.9507
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9379
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9238
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.9074
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.8890
Epoch 11/50
1/1 ━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 803 probe(s) 



Expression shape: (14, 162)
Latent shape: (14, 32)
Coefficients not estimable: 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 7 9 12 
Processed tissue: Minor Salivary Gland
Dimensions of adjusted data: (14, 162)
Skipping tissue Cervix-Ectocervix due to small sample size (9 samples).
Resaved normalized counts for Cells-EBV-transformed lymphocytes to ./data/processed/expression/readcounts_tmm_all/Cells-EBV-transformed lymphocytes_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0147
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9929
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.9777
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.9650
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.9513
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.9360
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9182
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.8987
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/st

...

Expression shape: (14, 258)
Latent shape: (14, 32)
Coefficients not estimable: 23 24 25 26 27 28 29 30 31 32 33 3 8 9 12 13 16 17 18 
Processed tissue: Adrenal Gland
Dimensions of adjusted data: (14, 258)
Skipping tissue Fallopian Tube due to small sample size (9 samples).
Resaved normalized counts for Brain-Cortex to ./data/processed/expression/readcounts_tmm_all/Brain-Cortex_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0272
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.9931
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.9705
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.9502
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.9271
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.9006
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.8706
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.8378
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.8037
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 258 probe(s) 



Expression shape: (14, 255)
Latent shape: (14, 32)
Coefficients not estimable: 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 4 
Processed tissue: Brain-Cortex
Dimensions of adjusted data: (14, 255)
Resaved normalized counts for Esophagus-Mucosa to ./data/processed/expression/readcounts_tmm_all/Esophagus-Mucosa_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0070
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.9942
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.9867
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.9779
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.9659
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.9506
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.9328
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 0.9124
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.8897
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.8657
Epoch 11/50
1/1 ━━━━━━━━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 255 probe(s) 



Expression shape: (14, 555)
Latent shape: (14, 32)
Coefficients not estimable: 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 2 3 6 
Processed tissue: Esophagus-Mucosa
Dimensions of adjusted data: (14, 555)
